<a href="https://colab.research.google.com/github/AngelGuadarrama/Consulta_DENUE/blob/main/extractor_denue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import csv
from os import listdir
import re
from ipywidgets import interact, interact_manual
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Extractor del Directorio Estadístico Nacional de Unidades Económicas (INEGI)

El objetivo es generar un programa, una base de datos o tabular, que le permita encontrar clientes para alianzas potenciales.

In [3]:
def capital(texto):
  words=texto.split(sep=' ')
  texto_cap=[word.capitalize() for word in words]
  resultado= " ".join(texto_cap)
  return resultado


def acentos(texto=""):
  words=texto.split(sep=' ')
  words=[re.sub(r"á", "a", word) for word in words]
  words=[re.sub(r"é", "e", word) for word in words]
  words=[re.sub(r"í", "i", word) for word in words]
  words=[re.sub(r"ó", "o", word) for word in words]
  words=[re.sub(r"ú", "u", word) for word in words]
  
  resultado= " ".join(words)
  
  return resultado



## Extractor y limpieza


*  Archivo compendio_total_contactable.txt (con |) (carpeta de lectura y escritura)

*  Mensaje con tamaño, nombre de columnas y columnas faltantes

*  Filtrar por telefono/correo, compendio_total_contactable.txt

* Quitar acentos, multiespacios y usar Capital()

In [ ]:
@interact_manual
def compendio_total(dir_int='/content/drive/MyDrive/Colab Notebooks/Extractor_DENUE/', dir_out='/content/drive/MyDrive/Colab Notebooks/Extractor_DENUE/'):
  global columnas_compendio
  
  csv_files=[file for file in listdir(dir_int) if file[-3:] == 'csv' and file[:5] == 'denue']
  #csv_files=["denue_inegi_16_.csv","denue_inegi_17_.csv","denue_inegi_13_.csv"]

  with open(dir_out+'compendio_total_contactable.txt', "w") as my_output_file: #Archivo txt
    columnas_compendio=[]

    for csv_file in csv_files: 
        print('Cargando archivo: '+csv_file)
        i=0 #Numero de filas (archivo inicial)
        j=0#Numero de filas (archivo limpio)

        with open(dir_int+csv_file, "r", encoding = "ISO-8859-1") as my_input_file: #Abre un archivo csv

            for row in csv.reader(my_input_file):

                if  i==0: #Encabezado
                   row.append('Ruta_Origen')
                   columnas=row
                   print('  Columnas: ', columnas)

                   if csv_files.index(csv_file) == 0: #Si es el primer archivo se agregan las columnas
                     columnas_compendio=columnas
                   else:
                     column_extra=list(set(columnas).difference(set(columnas_compendio)))
                     column_faltante=list(set(columnas_compendio).difference(set(columnas)))
                     #print('')
                     
                     print('Este archivo no contiene la(s) columna(s): ', column_extra)
                     print('Este archivo contiene ademas la(s) columna(s): ', column_faltante)
                     columnas_compendio+= column_extra

                   my_output_file.write("|".join(row)+'\n') #Separador y salto de linea

                else:
                   row.append(csv_file) #Ruta_Origen
                   
                   telefono= len( re.findall(r"\d{10}", row[columnas_compendio.index('telefono')]) ) != 0
                   correo= len( re.findall(r"@", row[columnas_compendio.index('correoelec')]) ) != 0

                   if i==1: print('  Primera Fila: ',row)

                   if telefono or correo:
                      row=[ capital(acentos( re.sub(r"\s+", " ",col ).lower() ) ) for col in row]
                      my_output_file.write("|".join(row)+'\n') #Separador y salto de linea

                      if j==1: print('  Primera Fila Limpia: ',row)
                      j+=1
                   
      
                i+=1  

        print('  Total de filas (archivo inicial): '+str(i)) 
        print('  Total de filas (archivo contactable): '+str(j)) 
        print('  Total de columnas: '+str(len(row)))  
        print('')
        print('')
    my_output_file.close()



  print('  Se ha creado el archivo compendio_total.txt exitosamente!')
  

## Analisis de Negocio

El objetivo es hacer un análisis de la base de datos contactable, para enunciar al área de negocio los giros principales por entidad federativa.

*	Descriptivo crudo de todos los registros que contenga la palabra Banco o cualquier otra palabra clave para un negocio

El descriptivo debe enunciar al menos nombre de actividad y personal ocupado.
¿Qué otras variables puedes ser reelevantes?

Otras variables relevantes podrian ser raz_soc, tipo_vial y la entidad

In [ ]:
banco=re.findall(r"Banco", "Banco inbursa")
print(banco == '')

False


In [54]:
@interact_manual
def cargar_compendio_limpio(dir_int='/content/drive/MyDrive/Colab Notebooks/Extractor_DENUE/compendio_total_contactable.txt', palabra='Banco', 
                            col_busqueda=['nom_estab', 'nombre_act'], col_cloud=['Nombre de Actividad', 'Nombre de Establecimiento']):
    global actividades
    global establecimientos

    with open(dir_int, "r") as file_clean:
        lines= file_clean.readlines()
        
        actividades=[]
        establecimientos=[]
        columnas_compendio=[]
        j=1

        for line in lines:
            row=line.split(sep="|")
            if row[0] == 'id': 
                columnas_compendio=row

            coincidencia= len( re.findall(palabra, row[columnas_compendio.index(col_busqueda)]) ) != 0

            if coincidencia :
                actividades.append(row[columnas_compendio.index('nombre_act')])
                establecimientos.append(row[columnas_compendio.index('nom_estab')])

                #print(j)
                #print('Nombre:', row[columnas_compendio.index('nom_estab')])
                #print('Actividad: ', row[columnas_compendio.index('nombre_act')])
                #print('Ocupacion: ', row[columnas_compendio.index('per_ocu')])
                #print('')
                j+=1              


        actividades_unicas = []
        [actividades_unicas.append(x) for x in actividades if x not in actividades_unicas]
        print('Total de negocios: ', str(j-1))
        print('')

        stop_words_sp = set(stopwords.words('spanish'))

        if col_cloud == 'Nombre de Actividad': cloud=WordCloud(stopwords= stop_words_sp, max_font_size=40, random_state=2).generate(' '.join(actividades))
        if col_cloud == 'Nombre de Establecimiento': cloud=WordCloud(stopwords= stop_words_sp, max_font_size=40, random_state=2).generate(' '.join(establecimientos))

        fig=plt.figure(figsize=(20,10))
        plt.imshow(cloud)
        plt.axis('off')
        plt.show()

interactive(children=(Text(value='/content/drive/MyDrive/Colab Notebooks/Extractor_DENUE/compendio_total_conta…

In [55]:
print('Numero de Actividades unicas: ', len(actividades_unicas))
actividades_unicas.sort()

print('')
print(*actividades_unicas, sep='\n')

Numero de Actividades unicas:  45

Administracion Publica En General
Agencias De Cobranza
Asesoria En Inversiones
Asociaciones Y Organizaciones Civiles
Asociaciones Y Organizaciones Laborales Y Sindicales
Asociaciones, Organizaciones Y Camaras De Productores, Comerciantes Y Prestadores De Servicios
Banca Central
Banca De Desarrollo
Banca Multiple
Bibliotecas Y Archivos Del Sector Publico
Cafeterias, Fuentes De Sodas, Neverias, Refresquerias Y Similares
Casas De Bolsa
Comercio Al Por Mayor De Cemento, Tabique Y Grava
Comercio Al Por Mayor De Maquinaria Y Equipo Para Otros Servicios Y Para Actividades Comerciales
Comercio Al Por Menor De Articulos Para La Limpieza
Comercio Al Por Menor De Blancos
Comercio Al Por Menor De Muebles Para El Hogar
Comercio Al Por Menor En Tiendas De Abarrotes, Ultramarinos Y Miscelaneas
Compañias De Seguros
Consultorios De Optometria
Corporativos
Estacionamientos Y Pensiones Para Vehiculos Automotores
Farmacias Sin Minisuper
Laboratorios Medicos Y De Diagnost


Suponiendo que no tuvieramos el apoyo del campo de nombre de actividad, sería importante observar que sí sabemos de nuestros datos, por su contenido:

*  Hacer una nube de bi-palabras y un Ngram (2 o 3) para entender que tipo de servicios se van a promover por estado (todos) usando solo el nombre del establecimiento

* En el ejemplo no se ha hecho limpieza ¿Qué tipo de limpiezas se pueden hacer?  (ngram, count)

In [39]:
vect= CountVectorizer(max_features=50, 
                      stop_words=stop_words_sp, 
                      ngram_range=(2,3))

X=vect.fit_transform(pd.Series(establecimientos)) 
top_50_bi_tri_grams=pd.DataFrame(X.toarray(), columns=vect.get_feature_names())
top_50_bi_tri_grams.sum().sort_values(ascending=False)

banco inbursa                 330
banco compartamos             143
inbursa walmart                78
banco inbursa walmart          78
banco inbursa sanborns         61
inbursa sanborns               61
banco bajio                    37
inbursa sams                   35
banco inbursa sams             35
banco wal                      28
wal mart                       27
banco wal mart                 27
institucion banca              26
banca multiple                 26
mart mexico                    25
mexico adelante                25
institucion banca multiple     25
wal mart mexico                24
mart mexico adelante           24
inbursa plaza                  20
banco inbursa plaza            20
banco inbursa bodega           19
bodega aurrera                 19
inbursa bodega                 19
inbursa bodega aurrera         19
banco forjadores               17
banco mexico                   17
bbva bancomer                  15
compartamos banco              15
banco arena   